In [22]:
import numpy as np
from scipy.stats import norm, multivariate_normal
from scipy.linalg import det, cho_factor

from gproc.generative import sample_at_x
from gproc.kernels import squared_exponential
from gproc.laplace import laplace_approximation_probit, chol_inverse

In [41]:
N = 50

# x = np.random.uniform(-1, 1, N).reshape(-1, 1) # Reshape to N x 1 matrix
x = np.random.normal(0, 10, N).reshape(-1, 1)

y, prob_y, f = sample_at_x(x, kernel_params={'lengthscale': 0.1, 'variance': 1.4})

In [42]:
gram = squared_exponential(x, x, lengthscale=0.1, variance=1.4)

# Work out determinant of kernel matrix
chol_gram = cho_factor(gram + 1e-5 * np.eye(gram.shape[0]), lower=True, check_finite=True)[0]
log_gram_det = 2 * np.sum(np.log(np.diagonal(chol_gram)))

inverse_gram = chol_inverse(gram)

In [43]:
laplace_mean, df_ll, laplace_cov, objective_history, converged = laplace_approximation_probit(y, inverse_gram)
print(f'Converged: {converged}')

# Get the log determinant and inverse of the approximate covariance matrix
chol_laplace_cov = cho_factor(laplace_cov + 1e-5 * np.eye(laplace_cov.shape[0]), lower=True, check_finite=True)[0]
log_laplace_cov_det = 2 * np.sum(np.log(np.diagonal(chol_laplace_cov)))
inverse_laplace_cov = chol_inverse(laplace_cov)

Converged: True


In [44]:
log_laplace_cov_det

-64.34206260392915

In [45]:
def log_joint(y, f, inverse_gram, log_gram_det):
    log_likelihood = np.sum(norm.logcdf(y * f))
    log_prior = -0.5 * ( y.shape[0] * np.log(2 * np.pi) - log_gram_det - f.dot(inverse_gram).dot(f) )
    return log_likelihood + log_prior

log_joint(y, f, inverse_gram, log_gram_det)

-55.76970702177828

In [46]:
def q_log_pdf(f, inverse_q_cov, log_q_det):
    return -0.5 * ( f.shape[0] * np.log(2 * np.pi) - log_q_det - f.dot(inverse_q_cov).dot(f) )

q_log_pdf(f, inverse_laplace_cov, log_laplace_cov_det)

-31.19906950426808

In [47]:
N_imp = 10
f_samples = np.random.multivariate_normal(laplace_mean, laplace_cov, N_imp)

log_joint(y, f_samples[0,:], inverse_gram, log_gram_det)

-66.64621153114616

In [48]:
def importance_sampler(y, x, q_mean, q_cov, N_imp, kernel_fcn=squared_exponential, kernel_params={}):
    # Get gram matrix
    gram = kernel_fcn(x, x, **kernel_params)
    
    # Inverse of gram matrix 
    inverse_gram = chol_inverse(gram)
    
    # Work out log-determinant of gram matrix
    chol_gram = cho_factor(gram + 1e-5 * np.eye(gram.shape[0]), lower=True, check_finite=True)[0]
    log_gram_det = 2 * np.sum(np.log(np.diagonal(chol_gram)))
    
    # Get the log determinant and inverse of the approximate covariance matrix
    chol_q_cov = cho_factor(q_cov + 1e-5 * np.eye(q_cov.shape[0]), lower=True, check_finite=True)[0]
    log_q_det = 2 * np.sum(np.log(np.diagonal(chol_q_cov)))
    inverse_q_cov = chol_inverse(q_cov)
    
    # Define log joint function
    def log_joint(y, f, inverse_gram, log_gram_det):
        log_likelihood = np.sum(norm.logcdf(y * f))
        log_prior = -0.5 * ( y.shape[0] * np.log(2 * np.pi) + log_gram_det + f.dot(inverse_gram).dot(f) )
        return log_likelihood + log_prior

    # Define log density of Normal approximation q
    def q_log_pdf(f, inverse_q_cov, log_q_det):
        return -0.5 * ( f.shape[0] * np.log(2 * np.pi) + log_q_det + f.dot(inverse_q_cov).dot(f) )

    # Sample latent functions from the normal approximation to 
    f_samples = np.random.multivariate_normal(q_mean, q_cov, N_imp)
    
    # Importance sum
    marg_approx = 0
    for i in range(N_imp):
        marg_approx += log_joint(y, f_samples[i, :], inverse_gram, log_gram_det) - q_log_pdf(f_samples[i, :], inverse_q_cov, log_q_det)
    
    return marg_approx/N_imp

In [51]:
importance_sampler(y, x, laplace_mean, laplace_cov, 1000)

-3358.8937567986864